In [4]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
import glob

In [5]:
y = ['0','1','2','3','4','5','6','7','8','9','A','a','B','b','C','c','D','d','E','e','F','f','G','g','H','h','I','i','J','j','K','k','L','l','M','m','N','n','O','o','P','p','Q','q','R','r','S','s','T','t','U','u','V','v','W','w','X','x','Y','y','Z','z'];
X = glob.glob("./font/*/*/*.png"); # array of image file paths